In [ ]:
import pandas as pd
import numpy as np

# Load all sheets from Forest Excel file
xls_forest = pd.ExcelFile(r'C:\Users\DAVID\Downloads\Bird_Monitoring_Data_FOREST.xlsx')
df_forest = pd.concat([
    xls_forest.parse(sheet).assign(Habitat_Type='Forest')
    for sheet in xls_forest.sheet_names
], ignore_index=True)

# Load all sheets from Grassland Excel file
xls_grass = pd.ExcelFile(r'C:\Users\DAVID\Downloads\Bird_Monitoring_Data_GRASSLAND.xlsx')
df_grass = pd.concat([
    xls_grass.parse(sheet).assign(Habitat_Type='Grassland')
    for sheet in xls_grass.sheet_names
], ignore_index=True)

# Merging both DF

df_combined = pd.concat([df_forest, df_grass], ignore_index=True)

# Date and Time formatting

df_combined['Date'] = pd.to_datetime(df_combined['Date'], errors='coerce')
df_combined['Start_Time'] = pd.to_datetime(df_combined['Start_Time'], format='%H:%M:%S', errors='coerce').dt.time
df_combined['End_Time'] = pd.to_datetime(df_combined['End_Time'], format='%H:%M:%S', errors='coerce').dt.time

# Missing values
# Fill NA for Sex
df_combined['Sex'] = df_combined['Sex'].fillna('Undetermined')

# Fill NA for Flyover_Observed
df_combined['Flyover_Observed'] = df_combined['Flyover_Observed'].fillna('FALSE')

# Fill Disturbance with "Unknown" if missing
df_combined['Disturbance'] = df_combined['Disturbance'].fillna('Unknown')

# Optionally drop rows with missing key identifiers
df_combined = df_combined.dropna(subset=['Scientific_Name', 'Common_Name'])

# Normalize category fields
df_combined['Flyover_Observed'] = df_combined['Flyover_Observed'].astype(str).str.upper().replace({'TRUE': 'Yes', 'FALSE': 'No'})
df_combined['Sex'] = df_combined['Sex'].replace({'M': 'Male', 'F': 'Female'}).str.capitalize()
df_combined['Location_Type'] = df_combined['Location_Type'].str.capitalize()

# Save Cleaned Data

df_combined.to_csv('C:\\Users\\DAVID\\Downloads\\cleaned_combined_data.csv', index=False)

print("Data cleaning and merging completed successfully.")
print("Shape of cleaned data:", df_combined.shape)

In [ ]:
# Temporal Analysis

In [ ]:
# Convert Date to datetime format
df_combined['Date'] = pd.to_datetime(df_combined['Date'], errors='coerce')

# Convert 'Start_Time' and 'End_Time' to time format
df_combined['Start_Time'] = pd.to_datetime(df_combined['Start_Time'], format='%H:%M:%S', errors='coerce').dt.time
df_combined['End_Time'] = pd.to_datetime(df_combined['End_Time'], format='%H:%M:%S', errors='coerce').dt.time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Load cleaned data
df = pd.read_csv('C:\\Users\\DAVID\\Downloads\\cleaned_combined_data.csv', parse_dates=['Date'])

# Display basic info
df.info()

In [ ]:
# Extract Year and Month from Date

In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['Month_Num'] = df['Date'].dt.month  # for sorting

In [ ]:
 # Line Plot: Year vs Observation Count

In [ ]:
yearly_counts = df.groupby('Year').size().reset_index(name='Observations')

fig = px.line(yearly_counts, x='Year', y='Observations', title='Yearly Bird Observations')
fig.show()

In [ ]:
#  Heatmap: Year x Month Sightings

In [ ]:
import calendar

In [ ]:
heatmap_data = df.groupby(['Year', 'Month_Num']).size().unstack().fillna(0)
heatmap_data.columns = [calendar.month_abbr[i] for i in heatmap_data.columns]

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=True, fmt='g')
plt.title("Monthly Bird Sightings by Year")
plt.xlabel("Month")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

In [ ]:
# Histogram: Start Time Distribution (Bird Activity)

In [ ]:
df['Start_Hour'] = pd.to_datetime(df['Start_Time'], errors='coerce').dt.hour

plt.figure(figsize=(10, 5))
sns.histplot(df['Start_Hour'].dropna(), bins=24, kde=False)
plt.title("Bird Observation Frequency by Hour of Day")
plt.xlabel("Hour (Start_Time)")
plt.ylabel("Number of Observations")
plt.grid(True)
plt.show()

In [ ]:
# Spatial Analysis

In [ ]:
# Top 10 Plots by Observation Count

In [ ]:
top_plots = df['Plot_Name'].value_counts().nlargest(10).reset_index()
top_plots.columns = ['Plot_Name', 'Observations']

fig = px.bar(top_plots, x='Plot_Name', y='Observations', title='Top 10 Active Plots')
fig.show()

In [ ]:
# Bar Chart: Location_Type vs Unique Species Count

In [ ]:
species_diversity = df.groupby('Location_Type')['Scientific_Name'].nunique().reset_index()
species_diversity.columns = ['Location_Type', 'Unique_Species']

fig = px.bar(species_diversity, x='Location_Type', y='Unique_Species', title='Species Diversity by Habitat Type')
fig.show()

In [ ]:
#  Pie Chart: Species Distribution in Top 5 Plots

In [ ]:
top_5_plots = df['Plot_Name'].value_counts().nlargest(5).index
top_species_in_top5 = df[df['Plot_Name'].isin(top_5_plots)]

species_distribution = top_species_in_top5['Scientific_Name'].value_counts().nlargest(5).reset_index()
species_distribution.columns = ['Scientific_Name', 'Count']

fig = px.pie(species_distribution, names='Scientific_Name', values='Count', 
             title='Species Distribution in Top 5 Plots')
fig.show()

In [ ]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# SPECIES ANALYSIS

In [ ]:
# Top 10 Most Observed Species (Scientific_Name)

In [ ]:
top_species = df['Scientific_Name'].value_counts().nlargest(10).reset_index()
top_species.columns = ['Scientific_Name', 'Observation_Count']

fig = px.bar(top_species, x='Scientific_Name', y='Observation_Count',
             title='Top 10 Most Observed Bird Species')
fig.show()

In [ ]:
# Observation Method (ID_Method) Distribution

In [ ]:
id_method_counts = df['ID_Method'].value_counts().reset_index()
id_method_counts.columns = ['ID_Method', 'Count']

fig = px.bar(id_method_counts, x='ID_Method', y='Count',
             title='Species Identification Methods Used')
fig.show()

In [ ]:
# Sex Distribution per Species (Top 5 Only for Clarity)

In [ ]:
# Filter top 5 species
top5_species = df['Scientific_Name'].value_counts().nlargest(5).index
sex_dist = df[df['Scientific_Name'].isin(top5_species)]

# Group and plot
sex_group = sex_dist.groupby(['Scientific_Name', 'Sex']).size().reset_index(name='Count')

fig = px.bar(sex_group, x='Scientific_Name', y='Count', color='Sex',
             title='Sex Distribution in Top 5 Species', barmode='group')
fig.show()

In [ ]:
# ENVIRONMENTAL FACTORS

In [ ]:
# Scatter Plot: Temperature vs Bird Count

In [ ]:
# Count observations per date & temperature
temp_count = df.groupby(['Date', 'Temperature']).size().reset_index(name='Bird_Count')

fig = px.scatter(temp_count, x='Temperature', y='Bird_Count',
                 title='Temperature vs Bird Count',
                 labels={'Bird_Count': 'Number of Observations'})
fig.show()

In [ ]:
# Boxplot: Humidity by Location Type

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Location_Type', y='Humidity', data=df)
plt.title('Humidity Distribution by Habitat Type')
plt.xlabel('Habitat')
plt.ylabel('Humidity (%)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Bar Chart: Wind Condition vs Observations

In [ ]:
wind_counts = df['Wind'].value_counts().reset_index()
wind_counts.columns = ['Wind_Condition', 'Count']

fig = px.bar(wind_counts, x='Wind_Condition', y='Count',
             title='Bird Observations by Wind Condition')
fig.show()

In [ ]:
# Bar Chart: Sky Condition vs Observations

In [ ]:
sky_counts = df['Sky'].value_counts().reset_index()
sky_counts.columns = ['Sky_Condition', 'Count']

fig = px.bar(sky_counts, x='Sky_Condition', y='Count',
             title='Bird Observations by Sky Condition')
fig.show()

In [ ]:
#  Disturbance Impact

In [ ]:
disturbance_counts = df['Disturbance'].value_counts().reset_index()
disturbance_counts.columns = ['Disturbance', 'Observation_Count']

fig = px.bar(disturbance_counts, x='Disturbance', y='Observation_Count',
             title='Impact of Disturbance on Bird Observations')
fig.show()

In [ ]:
# CONSERVATION RISK ANALYSIS

In [ ]:
# Pie Chart: % of Observations on PIF Watchlist

In [ ]:
# Normalize values to True/False
df['PIF_Watchlist_Status'] = df['PIF_Watchlist_Status'].fillna('FALSE').astype(str).str.upper()
df['PIF_Watchlist_Status'] = df['PIF_Watchlist_Status'].replace({'TRUE': 'Watchlist', 'FALSE': 'Non-Watchlist'})

# Count PIF status
pif_counts = df['PIF_Watchlist_Status'].value_counts().reset_index()
pif_counts.columns = ['Status', 'Count']

# Pie Chart
import plotly.express as px

fig = px.pie(pif_counts, names='Status', values='Count',
             title='% of Bird Observations by PIF Watchlist Status',
             color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [ ]:
# Bar Chart: Regional Stewardship Status by Species

In [ ]:
# Fill missing and clean
df['Regional_Stewardship_Status'] = df['Regional_Stewardship_Status'].fillna('FALSE').astype(str).str.upper()
df['Regional_Stewardship_Status'] = df['Regional_Stewardship_Status'].replace({'TRUE': 'Stewardship', 'FALSE': 'General'})

# Group by species and stewardship
stewardship_species = df[df['Regional_Stewardship_Status'] == 'Stewardship']
species_stewardship = stewardship_species['Scientific_Name'].value_counts().nlargest(10).reset_index()
species_stewardship.columns = ['Scientific_Name', 'Observation_Count']

# Bar Chart
fig = px.bar(species_stewardship, x='Scientific_Name', y='Observation_Count',
             title='Top 10 Stewardship Species by Observations')
fig.show()

In [ ]:
# Bar Chart: AOU_Code vs Observation Count

In [ ]:
# Group by AOU code
aou_counts = df['AOU_Code'].value_counts().nlargest(15).reset_index()
aou_counts.columns = ['AOU_Code', 'Observation_Count']

# Bar Chart
fig = px.bar(aou_counts, x='AOU_Code', y='Observation_Count',
             title='Top 15 AOU Codes by Observation Count')
fig.show()

In [ ]:
import mysql.connector

# Connect to your MySQL database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Melmir@123',
    database='david'
)
cursor = conn.cursor()

In [ ]:
# Create bird_observations Table

In [ ]:
create_table_query = """
CREATE TABLE IF NOT EXISTS bird_observations (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Admin_Unit_Code VARCHAR(50),
    Sub_Unit_Code VARCHAR(50),
    Site_Name VARCHAR(255),
    Plot_Name VARCHAR(255),
    Location_Type VARCHAR(50),
    Year INT,
    Date DATE,
    Start_Time TIME,
    End_Time TIME,
    Observer VARCHAR(100),
    Visit INT,
    Interval_Length VARCHAR(50),
    ID_Method VARCHAR(100),
    Distance VARCHAR(50),
    Flyover_Observed VARCHAR(10),
    Sex VARCHAR(20),
    Common_Name VARCHAR(255),
    Scientific_Name VARCHAR(255),
    AcceptedTSN VARCHAR(50),
    NPSTaxonCode VARCHAR(50),
    AOU_Code VARCHAR(20),
    PIF_Watchlist_Status VARCHAR(20),
    Regional_Stewardship_Status VARCHAR(20),
    Temperature FLOAT,
    Humidity FLOAT,
    Sky VARCHAR(100),
    Wind VARCHAR(100),
    Disturbance VARCHAR(255),
    Initial_Three_Min_Cnt INT,
    Habitat_Type VARCHAR(50)
);
"""

cursor.execute(create_table_query)
conn.commit()

In [ ]:
df = pd.read_csv('C:\\Users\\DAVID\\Downloads\\cleaned_combined_data.csv', low_memory=False)

In [ ]:
df = pd.read_csv('C:\\Users\\DAVID\\Downloads\\cleaned_combined_data.csv', dtype={'Sub_Unit_Code': str, 'AOU_Code': str, 'Initial_Three_Min_Cnt': float}, low_memory=False)

In [ ]:
columns_needed = [
    'Admin_Unit_Code', 'Sub_Unit_Code', 'Site_Name', 'Plot_Name', 'Location_Type',
    'Year', 'Date', 'Start_Time', 'End_Time', 'Observer', 'Visit',
    'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
    'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'NPSTaxonCode', 'AOU_Code',
    'PIF_Watchlist_Status', 'Regional_Stewardship_Status', 'Temperature', 'Humidity',
    'Sky', 'Wind', 'Disturbance', 'Initial_Three_Min_Cnt', 'Habitat_Type'
]

# Filter and reorder your dataframe
df = df[columns_needed]
df = df.where(pd.notnull(df), None)  # Replace NaNs with None

In [ ]:
insert_query = """
INSERT INTO bird_observations (
    Admin_Unit_Code, Sub_Unit_Code, Site_Name, Plot_Name, Location_Type, Year, Date,
    Start_Time, End_Time, Observer, Visit, Interval_Length, ID_Method, Distance,
    Flyover_Observed, Sex, Common_Name, Scientific_Name, AcceptedTSN, NPSTaxonCode,
    AOU_Code, PIF_Watchlist_Status, Regional_Stewardship_Status, Temperature, Humidity,
    Sky, Wind, Disturbance, Initial_Three_Min_Cnt, Habitat_Type
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
inserted = 0
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query, tuple(row))
        inserted += 1
    except Exception as e:
        print(f"❌ Row {i} failed: {e}")

conn.commit()
print(f"✅ {inserted} rows inserted successfully.")